### Hanoi mit Klassen
Vergleiche mit dem Notebook `Tuerme_von_Hanoi.ipynb` der Lektion 12.  
Beachte: `event_handler` ist ein Attribut der Instanz. Dieses Attribut speichert eine Funktion, die
von den Methoden `new_game` und `move_disk` aufgerufen wird. 
Diese Funktion soll den aktuellen Spielzustand darstellen, hier die 3 Stapel. 

Der Default Eventhandler ist die Funktion `print`.
Wir geben einfach die Argumente aus, mit denen die Funktion `event_handler` aufgerufen wird.

In [1]:
#%%file game.py
class Game:
    def __init__(self):
        self.ndisks = 4
        self.stacks = None
        self.event_handler = print

    def new_game(self):
        self.stacks = [list(range(self.ndisks))[::-1], [], []]
        self.event_handler('new_game', self.stacks)

    def move_disk(self, src, dst):
        if (not self.stacks[src] or
           (self.stacks[dst] and self.stacks[dst][-1] < self.stacks[src][-1])):
            return
        disk = self.stacks[src].pop()
        self.stacks[dst].append(disk)
        self.event_handler('update_stacks', self.stacks)

    def __repr__(self):
        return str(self.stacks)

In [2]:
game = Game()
game

None

In [3]:
game.new_game()

new_game [[3, 2, 1, 0], [], []]


In [4]:
game.move_disk(0, 2)

update_stacks [[3, 2, 1], [], [0]]


In [5]:
game

[[3, 2, 1], [], [0]]

In [6]:
#%%file view.py
import widgets_helpers
from ipycanvas import Canvas
from IPython.display import display


class CanvasView:
    positions = [50, 150, 250]
    disk_widths = [30, 50, 70, 90]
    disk_height = 10
    colors = ['brown', 'teal', 'blue', 'purple']

    def __init__(self, game):
        self.game = game
        self.canvas = widgets_helpers.new_canvas()
        self.game.event_handler = self.callback

    def draw_stack(self, stack, pos):
        h = self.disk_height
        for i, disk in enumerate(stack):
            self.canvas.fill_style = self.colors[disk]
            w = self.disk_widths[disk]
            self.canvas.fill_rect(pos - w/2,
                                  self.canvas.height-(i+1)*h,
                                  w,
                                  h)

    def draw_stacks(self, stacks):
        self.canvas.clear()
        for pos, stack in zip(self.positions, stacks):
            self.draw_stack(stack, pos)

    def callback(self, event, data):
        self.draw_stacks(data)

    def _ipython_display_(self):
        display(self.canvas)

In [7]:
view = CanvasView(game)
view

Canvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

In [8]:
# Methode draw_stacks testen
stacks = [[2, 1], [3, 2, 1], [1, 2, 3]]
view.draw_stacks(stacks)

In [9]:
game.new_game()

In [10]:
game.move_disk(0, 1)

In [29]:
#%%file controller.py
from IPython.display import display
from canvas_callbacks import remove_all_callbacks
from widgets_helpers import new_output


class Controller:
    def __init__(self, game, canvas, remove_callbacks=True, debug=True):
        self.game = game
        self.canvas = canvas
        if debug:
            self.out = new_output()

        self.src = None
        self.dst = None

        if remove_all_callbacks:
            remove_all_callbacks(self.canvas)
        if debug:
            self.on_key_down = self.out.capture()(self.on_key_down)

        self.canvas.on_key_down(self.on_key_down)

    def on_key_down(self, key, *flags):
        if key in '123' and self.src is None:
            self.src = int(key) - 1
            return
        elif key in '123':
            self.dst = int(key) - 1
            self.game.move_disk(self.src, self.dst)
        if key == 'n':
            self.game.new_game()
        self.src = None

    def _ipython_display_(self):
        display(self.canvas, self.out)

Overwriting controller.py


In [20]:
controller = Controller(game, view.canvas)
controller

Canvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [21]:
game.new_game()

In [15]:
game.move_disk(0, 1)

In [16]:
controller.game.new_game()

In [17]:
controller.game.move_disk(0, 1)

In [22]:
controller.on_key_down()

In [23]:
controller.out.clear_output()

### Aufgabe
Erweitere die Klasse `Controller` um Methoden
`on_mouse_down` und `on_mouse_up`, welche das Verschieben der Scheiben mit der Maus 
m&ouml;glich machen sollen. Orientiere dich an der Maussteuerung im Notebook `Tuerme_von_Hanoi.ipynb` der Lektion 12.
Passe dann die `__init__`-Methode entsprechend an.



```python

    def __init__(self, game, canvas, remove_callbacks=True, debug=True):
        ...
        if debug:
            self.on_key_down = self.out.capture()(self.on_key_down)
            self.on_mouse_down = self.out.capture()(self.on_mouse_down)
            self.on_mouse_up = self.out.capture()(self.on_mouse_up)
        
        self.canvas.on_key_down(self.on_key_down)
        self.canvas.on_mouse_down(self.on_mouse_down)
        self.canvas.on_mouse_up(self.on_mouse_up)
        ...

    def on_mouse_down(self, x, y):
        ...
    
    def on_mouse_up(self, x, y):
        ...
```